# Homework 4: Association Rule Mining

- **100 points [8% of your final grade]**
- **Due Sunday, April 28 by 11:59pm**

- *Goals of this homework:* implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw4.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw4.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

In this assignment, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [1]:
import numpy as np
from itertools import combinations
from tqdm.notebook import tqdm
import pandas as pd

user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id watched the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been watched by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user movie-watching behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will explain the process of the method.**

**Hint: If you implement your algorithm correctly, you should be able to see intermediate result as:**
- Candidates of length 1 count: 408
- After Pruning count: 21
- Candidates of length 2 count: 210
- After Pruning 2 count: 36
- Candidates of length 3 count: 55
- After Pruning 3 count: 12
- Candidates of length 4 count: 1
- After Pruning 4 count: 0

In [2]:
# Write your code
# include all the helpful print statements
# when you run this block, we want to see all of your intermediate steps
# you can save the rules you discover for printing in the following cells (this will help us grade by keeping these separate)

def load_data(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    transactions = {}
    for user, movie in data:
        transactions.setdefault(user, list()).append(int(movie))
    return list(transactions.values())

def generate_candidate(data, i):
    # Flatten the list of transactions to find all unique items
    unique_items = set()
    for transaction in data:
        for item in transaction:
            unique_items.add(item)

    # Generate all combinations of the unique items of length 'i'
    candidates = []
    for combination in combinations(unique_items, i):
        # Convert each combination from a tuple to a set
        candidate = set(combination)
        candidates.append(candidate)

    return candidates

def calculate_support(transactions, candidate):
    """
    Calculate the support for a single candidate itemset.
    """
    count = 0
    for transaction in transactions: 
        is_subset = True
        for item in candidate:
            if item not in transaction:
                is_subset = False
                break
        if is_subset:
            count += 1  
    support = count / len(transactions)
    return support

def filter_candidates_by_support(transactions, candidates, min_support):
    """
    Filter candidate itemsets that have a support greater than or equal to the given minimum support.
    """
    frequent_itemsets = []
    for candidate in candidates:
        support = calculate_support(transactions, candidate)
        if support >= min_support:
            frequent_itemsets.append(candidate)
    return frequent_itemsets

def has_infrequent_subset(candidate, all_frequent_itemsets):
    for i in range(1, len(candidate)):
        subsets = combinations(candidate, i)
        for subset in subsets:
            frozen_subset = frozenset(subset)
            if frozen_subset not in all_frequent_itemsets:
                return True
    return False

def apriori_algorithm(data, x, min_support):
    all_frequent_itemsets = []

    # This will hold the frequent itemsets found in the last iteration
    last_frequent_itemsets = []

    for i in range(1, x+1):
        print(f"Generating candidates of length {i}")

        # Generate candidates of the current length
        if (i==1):
            candidates = generate_candidate(data, i)
        else:
            candidates = generate_candidate(frequent_itemsets, i)
            #print(f"Candidates of length {i} count: {len(candidates)}")

        # Initialize a new list to store frequent candidates
        frequent_candidates = []

        # Filter out candidates that contain infrequent subsets if i > 1
        for candidate in candidates:
            if has_infrequent_subset(candidate, all_frequent_itemsets) == False:
                frequent_candidates.append(candidate)

        # Calculate the support for each candidate and filter them based on the support threshold
        frequent_itemsets = filter_candidates_by_support(data, frequent_candidates, min_support)
        print(f"Candidates of length {i} count: {len(frequent_candidates)}")
        print("After Pruning count:",len(frequent_itemsets))
        
        
        if len(frequent_itemsets)==0:
            print(f"No frequent itemsets of length {i}. Ending the algorithm.")
            break

        #print(f"Frequent itemsets of length {i}: {frequent_itemsets}")
        # Prepare for the next iteration
        last_frequent_itemsets = frequent_itemsets
        all_frequent_itemsets.extend(frequent_itemsets)
        
    #print(f"All frequent itemsets: {all_frequent_itemsets}")
    return all_frequent_itemsets

x = 4
min_support = 0.2
# data = [[1,2,3,4,5],[4,6]]
data = load_data("movie_rated.txt")
frequent_itemsets = apriori_algorithm(data, x, min_support)

test = [[1,2,3,4,5],[4,6]]
# print((generate_candidate(test, 4)))

Generating candidates of length 1
Candidates of length 1 count: 408
After Pruning count: 21
Generating candidates of length 2
Candidates of length 2 count: 210
After Pruning count: 36
Generating candidates of length 3
Candidates of length 3 count: 55
After Pruning count: 12
Generating candidates of length 4
Candidates of length 4 count: 1
After Pruning count: 0
No frequent itemsets of length 4. Ending the algorithm.


Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: if you implement the algorith correctly, you will find 14 rules in total.**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [3]:
# Write your code to print out the rules
frequent_itemsets = [itemset for itemset in frequent_itemsets if len(itemset) > 1]
print(frequent_itemsets)

[{260, 527}, {2987, 260}, {260, 1197}, {260, 1221}, {858, 260}, {480, 260}, {260, 1127}, {1387, 260}, {260, 2028}, {1265, 260}, {260, 1270}, {377, 260}, {1197, 527}, {858, 527}, {480, 527}, {2028, 527}, {1265, 527}, {1270, 527}, {2987, 1197}, {2987, 1270}, {858, 1197}, {480, 1197}, {2028, 1197}, {1265, 1197}, {1197, 1270}, {858, 1221}, {480, 858}, {858, 2028}, {858, 1270}, {480, 2028}, {480, 1265}, {480, 1270}, {480, 377}, {1265, 2028}, {2028, 1270}, {1265, 1270}, {480, 260, 1197}, {480, 260, 2028}, {480, 260, 1270}, {858, 260, 1221}, {260, 1197, 2028}, {1265, 260, 1197}, {260, 1197, 1270}, {260, 2028, 527}, {1270, 260, 2028}, {260, 1270, 527}, {1265, 260, 1270}, {1265, 1197, 1270}]


In [4]:
movies_df = pd.read_csv('movies.csv')
movie_name = []
movie_id = []
for i in range(len(movies_df)):
    movie_name.append(movies_df.iloc[i,1])
    movie_id.append(movies_df.iloc[i,0])
    
movie_dict = dict(zip(movie_id, movie_name))
# print(movie_dict)

In [5]:
def generate_rules(record,frequent_itemsets, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
         for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = set(antecedent)
                consequent = itemset - antecedent
                conf = calculate_support(record, itemset) / calculate_support(record, antecedent)
                if conf >= min_confidence:
                    rules.append((antecedent, consequent))           
    return rules

def rules_to_names(rules, movie_dict):
    rules_with_names = []
    for antecedent_ids, consequent_ids in rules:
        antecedent_names = [movie_dict[id] for id in antecedent_ids if id in movie_dict]
        consequent_names = [movie_dict[id] for id in consequent_ids if id in movie_dict]
        rules_with_names.append((antecedent_names, consequent_names))
        print(antecedent_names, "-->", consequent_names)
    
    return rules_with_names


r = generate_rules(data, frequent_itemsets, 0.8)
rules_with_movie_names = rules_to_names(r, movie_dict)

['Jaws (1975)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Godfather: Part II, The (1974)'] --> ['Godfather, The (1972)']
['Jurassic Park (1993)', 'Princess Bride, The (1987)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Jurassic Park (1993)', 'Saving Private Ryan (1998)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Jurassic Park (1993)', 'Back to the Future (1985)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Godfather, The (1972)', 'Godfather: Part II, The (1974)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Star Wars: Episode IV - A New Hope (1977)', 'Godfather: Part II, The (1974)'] --> ['Godfather, The (1972)']
['Saving Private Ryan (1998)', 'Princess Bride, The (1987)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Star Wars: Episode IV - A New Hope (1977)', 'Princess Bride, The (1987)'] --> ['Back to the Future (1985)']
['Princess Bride, The (1987)', 'Back to the Future (1985)'] --> ['Star Wars: Episode IV - A New Hope (1977)']
['Sa